In [1]:
import pandas as pd
import requests
import json
import time

In [2]:
resp = requests.get('https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=599.0')

In [3]:
if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))
for item in resp.json():
    print(item)

1
['5990']
None
[['599.0', ' Urinary tract infection, site not specified']]


In [4]:
files = resp.json()
print (files)

[1, ['5990'], None, [['599.0', ' Urinary tract infection, site not specified']]]


In [5]:
len(files)

4

In [6]:
files[3][0][1]

' Urinary tract infection, site not specified'

# ⇪ Ah Ha !

> Now let's execute that for the list of codes I'm working on

- - -

In [7]:
codes = [580.0, 581.0, 582.0, 583.0, 584.0, 585.0, 586.0, 587.0, 588.0, 589.0, 590.0, 591.0, 592.0, 593.0, 594.0, 595.0, 596.0, 597.0, 598.0, 599.0, 600.0, 601.0, 602.0, 603.0, 604.0, 605.0, 606.0, 607.0, 608.0, 610.0, 611.0, 616.0, 612.0, 614.0, 615.0, 617.0, 618.0, 619.0, 620.0, 621.0, 622.0, 623.0, 624.0, 625.0, 626.0, 627.0, 628.0, 629.0]

In [8]:
codes[4]

584.0

In [9]:
arr = []
for c in codes:
    code = 'https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=' + str(c)
    resp = requests.get(code)
    if resp.status_code != 200:
        # This means something went wrong.
        raise ApiError('GET /tasks/ {}'.format(resp.status_code))
    elif resp.json()[0] == 0:
        # assuming that means that this ICD code has an .XX higher up, in other words:
        # "584.0" DOES NOT exist, but...
        # "584.3" DOES exist
        # so, strip out the period, get the multiple item list, and take the first one in it
        noperiod = str.split(str(c),'.')[0]
#        print (noperiod)
        newcode = 'https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=' + noperiod
        manyresp = requests.get(newcode)
        arr.append(manyresp.json()[3][0][1])
        print (newcode)
        print (manyresp.json()[3][0][1])
    else:
        arr.append(resp.json()[3][0][1])
        print (code)
        print (resp.json()[3][0][1])

https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=580.0
 Acute glomerulonephritis with lesion of proliferative glomerulonephritis
https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=581.0
 Nephrotic syndrome with lesion of proliferative glomerulonephritis
https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=582.0
 Chronic glomerulonephritis with lesion of proliferative glomerulonephritis
https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=583.0
 Nephritis and nephropathy, not specified as acute or chronic, with lesion of proliferative glomerulonephritis
https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=584
 Acute kidney failure with lesion of tubular necrosis
https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=585
 Chronic kidney disease, Stage II (mild)
https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search?terms=586
 Renal failure, unspecified
https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/searc

In [10]:
len(arr)

48

In [11]:
len(codes)

48

# ⇪ Yes!

> Now write them to a file so you can merge them in another dataframe

In [12]:
icd_codes = list(zip(codes,arr))
icd_codes

[(580.0,
  ' Acute glomerulonephritis with lesion of proliferative glomerulonephritis'),
 (581.0,
  ' Nephrotic syndrome with lesion of proliferative glomerulonephritis'),
 (582.0,
  ' Chronic glomerulonephritis with lesion of proliferative glomerulonephritis'),
 (583.0,
  ' Nephritis and nephropathy, not specified as acute or chronic, with lesion of proliferative glomerulonephritis'),
 (584.0, ' Acute kidney failure with lesion of tubular necrosis'),
 (585.0, ' Chronic kidney disease, Stage II (mild)'),
 (586.0, ' Renal failure, unspecified'),
 (587.0, ' Renal sclerosis, unspecified'),
 (588.0, ' Renal osteodystrophy'),
 (589.0, ' Unilateral small kidney'),
 (590.0, ' Chronic pyelonephritis without lesion of renal medullary necrosis'),
 (591.0, ' Hydronephrosis'),
 (592.0, ' Calculus of kidney'),
 (593.0, ' Nephroptosis'),
 (594.0, ' Calculus in diverticulum of bladder'),
 (595.0, ' Acute cystitis'),
 (596.0, ' Bladder neck obstruction'),
 (597.0, ' Urethral abscess'),
 (598.0, ' Uret

In [14]:
df = pd.DataFrame(icd_codes, columns=['ICD_9_CODE','DIAGNOSIS_DESCRIPTION'])
df

,ICD_9_CODE,DIAGNOSIS_DESCRIPTION
0,580.0,Acute glomerulonephritis with lesion of proli...
1,581.0,Nephrotic syndrome with lesion of proliferati...
2,582.0,Chronic glomerulonephritis with lesion of pro...
3,583.0,"Nephritis and nephropathy, not specified as a..."
4,584.0,Acute kidney failure with lesion of tubular n...
5,585.0,"Chronic kidney disease, Stage II (mild)"
6,586.0,"Renal failure, unspecified"
7,587.0,"Renal sclerosis, unspecified"
8,588.0,Renal osteodystrophy
9,589.0,Unilateral small kidney


In [15]:
df.to_csv('/home/jupyter/data/cms/uti-icd.csv',index=None)